In [1]:
from pikov import pikov

In [2]:
gamekitty = pikov.Pikov.open('gamekitty.pikov')

In [3]:
clip = gamekitty.get_clip(6)

In [4]:
clip.frames[0].image

In [5]:
gamekitty.get_image('md5-992b3e8f73a3d10574c9cc2a9e320475')

In [6]:
gamekitty.get_image('md5-daa933078cce1a7374a20e52b1ddef8a')

In [ ]:
import importlib
importlib.reload(pikov)